Dataset cleaning 

In [ ]:
import pandas as pd

# Load the full dataset
file_path = "/kaggle/input/guidewire-main-ds/dataSynthetic.csv"  # Update with the actual dataset path
df = pd.read_csv(file_path)

# Convert 'Timestamp' column to datetime format and sort the dataset
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%Y-%m-%d %H:%M:%S')
df = df.sort_values(by='Timestamp')

# Drop columns that are empty or have only zeros
columns_to_drop = ['Pod Reason', 'Memory Usage (Swap) (MB)']
df.drop(columns=columns_to_drop, inplace=True)

# Handling missing values for the remaining columns
for col in df.columns:
    if df[col].isnull().sum() > 0:  # Only process columns with missing values
        if df[col].dtype == 'object':  # Categorical columns
            df[col].fillna(df[col].mode()[0], inplace=True)
        else:  # Numerical columns
            df[col].fillna(df[col].median(), inplace=True)

# Save the cleaned dataset
df.to_csv("/kaggle/working/cleaned_dataset.csv", index=False)

print("Data cleaning completed. Sorted by Timestamp, missing values handled, and redundant columns removed.")


Dataset info displaying code along with saving first 10 rows as a csv 

In [9]:
import pandas as pd

# Load the dataset
file_path = "/kaggle/input/clean-dataset/cleaned_dataset.csv"  # Change this to your dataset path
df = pd.read_csv(file_path)

# Determine column types (Numerical/Categorical)
column_types = {}
for col in df.columns:
    if df[col].dtype in ['int64', 'float64']:  # Numerical types
        column_types[col] = "Numerical"
    else:
        column_types[col] = "Categorical"

# Print column names and their types
print("Column Name | Content Type")
print("-" * 30)
for col, dtype in column_types.items():
    print(f"{col} | {dtype}")

# Save the first 10 rows into a new CSV file
output_path = "/kaggle/working/cleaned_first_10_rows.csv"
df.head(10).to_csv(output_path, index=False)

print(f"\nFirst 10 rows saved to: {output_path}")


Column Name | Content Type
------------------------------
Timestamp | Categorical
Pod Name | Categorical
CPU Usage (%) | Numerical
Memory Usage (%) | Numerical
Pod Status | Categorical
Pod Restarts | Numerical
Ready Containers | Numerical
Total Containers | Numerical
Pod Event Type | Categorical
Pod Event Reason | Categorical
Pod Event Age | Categorical
Pod Event Source | Categorical
Pod Event Message | Categorical
Node Name | Categorical
Event Reason | Categorical
Event Age | Categorical
Event Source | Categorical
Event Message | Categorical
Memory Usage (MB) | Numerical
Active Memory Requests (MB) | Numerical
Memory Requests (%) | Numerical
Active Memory Limits (MB)emory Limits Utilization (%) | Numerical
Memory Usage (MB).1 | Numerical
Memory Usage (Cache) (MB) | Numerical
Network Receive Bytes | Numerical
Network Transmit Bytes | Numerical
Network Receive Packets (p/s) | Numerical
Network Transmit Packets (p/s) | Numerical
Network Receive Packets Dropped (p/s) | Numerical
Network T

code to retain only 19 imp cols and add some feature engineering 

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load the dataset
file_path = "/kaggle/input/clean-dataset/cleaned_dataset.csv"  # Update with your actual file path
df = pd.read_csv(file_path)

# Retain only the 19 important columns
selected_columns = [
    # Failure-related columns (10)
    "Pod Status", "Pod Event Type", "Pod Event Reason", 
    "CPU Usage (%)", "Memory Usage (%)", "Pod Restarts", 
    "Network Receive Packets Dropped (p/s)", "Network Transmit Packets Dropped (p/s)", 
    "FS Reads Total (MB)", "FS Writes Total (MB)",

    # Other important columns (9)
    "Timestamp", "Pod Name", "Node Name", 
    "Ready Containers", "Total Containers", "Pod Event Age", 
    "Pod Event Source", "Event Age", "Event Source"
]

df = df[selected_columns]

### 1. Encoding Categorical Variables
# Label encoding for categorical columns
label_encoders = {}
for col in ["Pod Status", "Pod Event Type", "Pod Event Reason", "Pod Event Source", "Event Source"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders for later use (if needed)

### 2. Timestamp Decomposition
df["Timestamp"] = pd.to_datetime(df["Timestamp"])  # Convert to datetime format
df["Hour"] = df["Timestamp"].dt.hour
df["Day"] = df["Timestamp"].dt.day
df["Month"] = df["Timestamp"].dt.month
df["Weekday"] = df["Timestamp"].dt.weekday  # 0 = Monday, 6 = Sunday

# Drop the original Timestamp column
df.drop(columns=["Timestamp"], inplace=True)

### 3. Creating Binary Failure Labels
df["Pod_Status_Failure"] = (df["Pod Status"] != 0).astype(int)  # Assuming 0 = Running
df["Pod_Event_Failure"] = (df["Pod Event Type"] != 0).astype(int)  # Assuming 0 = Normal
df["Pod_Event_Reason_Failure"] = (df["Pod Event Reason"] == 1).astype(int)  # Assuming BACKOFF is 1
df["Resource_Failure"] = ((df["Pod Event Reason"] == 2) | (df["CPU Usage (%)"] > 2.0) | (df["Memory Usage (%)"] > 2.0)).astype(int)  # Assuming OOMKILLED is 2
df["Pod_Restart_Failure"] = (df["Pod Restarts"] > 0).astype(int)
df["Network_Failure"] = ((df["Network Receive Packets Dropped (p/s)"] > 1.5) | (df["Network Transmit Packets Dropped (p/s)"] > 1.5)).astype(int)
df["Disk_Failure"] = ((df["FS Reads Total (MB)"] > 2.0) | (df["FS Writes Total (MB)"] > 2.0)).astype(int)

### 4. Normalization/Scaling
scaler = StandardScaler()
num_cols = ["CPU Usage (%)", "Memory Usage (%)", "Pod Restarts", 
            "Network Receive Packets Dropped (p/s)", "Network Transmit Packets Dropped (p/s)", 
            "FS Reads Total (MB)", "FS Writes Total (MB)"]

df[num_cols] = scaler.fit_transform(df[num_cols])

# Save the transformed dataset
df.to_csv("/kaggle/working/transformed_dataset.csv", index=False)
print("Feature engineering completed. Transformed dataset saved as 'transformed_dataset.csv'.")


<ipython-input-10-69ac2767af87>:34: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Timestamp"] = pd.to_datetime(df["Timestamp"])  # Convert to datetime format


Feature engineering completed. Transformed dataset saved as 'transformed_dataset.csv'.
